In [18]:
import os
import requests
import pandas as pd
import numpy as np
import scipy.stats as st
import pytz
import datetime as dt

In [4]:
api_key = os.environ['CYAN_KEY']

In [3]:
api_key

'0b2e4bea2e3a07f762beb59ba9640ce7'

In [14]:
baseurl = 'https://api.darksky.net/forecast/'
url = baseurl + api_key + '/37.8,-122.2,2019-08-01T08:00:00?exclude=current,flags'

In [15]:
forecast = requests.get(url).json()
forecast

{'latitude': 37.8,
 'longitude': -122.2,
 'timezone': 'America/Los_Angeles',
 'currently': {'time': 1564671600,
  'summary': 'Mostly Cloudy',
  'icon': 'partly-cloudy-day',
  'precipIntensity': 0,
  'precipProbability': 0,
  'temperature': 58.22,
  'apparentTemperature': 58.22,
  'dewPoint': 54.09,
  'humidity': 0.86,
  'pressure': 1015.94,
  'windSpeed': 7.41,
  'windGust': 8.39,
  'windBearing': 216,
  'cloudCover': 0.78,
  'uvIndex': 1,
  'visibility': 10,
  'ozone': 302.8},
 'hourly': {'summary': 'Partly cloudy throughout the day.',
  'icon': 'partly-cloudy-day',
  'data': [{'time': 1564642800,
    'summary': 'Clear',
    'icon': 'clear-night',
    'precipIntensity': 0,
    'precipProbability': 0,
    'temperature': 58.05,
    'apparentTemperature': 58.05,
    'dewPoint': 54.49,
    'humidity': 0.88,
    'pressure': 1014.23,
    'windSpeed': 8.39,
    'windGust': 9.25,
    'windBearing': 206,
    'cloudCover': 0.03,
    'uvIndex': 0,
    'visibility': 10,
    'ozone': 300.5},
   {'

In [21]:
forecast['hourly']['data'][0]['wind']

{'time': 1564642800,
 'summary': 'Clear',
 'icon': 'clear-night',
 'precipIntensity': 0,
 'precipProbability': 0,
 'temperature': 58.05,
 'apparentTemperature': 58.05,
 'dewPoint': 54.49,
 'humidity': 0.88,
 'pressure': 1014.23,
 'windSpeed': 8.39,
 'windGust': 9.25,
 'windBearing': 206,
 'cloudCover': 0.03,
 'uvIndex': 0,
 'visibility': 10,
 'ozone': 300.5}

In [25]:
forecast['hourly']['data'][10]['windGust']

8.34

In [26]:
list1 = []
list1.append(forecast['hourly']['data'][10]['windBearing'])
list1.append(forecast['hourly']['data'][10]['windGust'])
list1.append(forecast['hourly']['data'][10]['precipIntensity'])
list1

[225, 8.34, 0]

In [29]:
model_df = pd.read_csv('../../score_model_data/score_model.csv')
model_df.head()

,Unnamed: 0,course_id,cumgame_coeff,intercept,mse,n_value,nomatch,player,precip_coeff,prediction_score,wind_coeff
0,0,0,NaN,NaN,NaN,NaN,1.0,Ben Horst,NaN,NaN,NaN
1,1,1,NaN,NaN,NaN,NaN,1.0,Ben Horst,NaN,NaN,NaN
2,2,2,-0.0,68.111111,16.320988,9.0,NaN,Ben Horst,0.0,0.0,0.0
3,3,0,NaN,NaN,NaN,NaN,1.0,Bob Montes,NaN,NaN,NaN
4,4,1,0.0,52.600000,6.640000,5.0,NaN,Bob Montes,0.0,0.0,-0.0


In [30]:
model_df.mean()

Unnamed: 0          724.000000
course_id             1.000000
cumgame_coeff         0.047041
intercept            61.017207
mse                   9.240555
n_value               9.888060
nomatch               1.000000
precip_coeff          0.061385
prediction_score      0.302903
wind_coeff            0.170194
dtype: float64

In [31]:
model_df.mean().loc['intercept']

61.01720696991604

In [42]:
predict_df = model_df.loc[(model_df['player'] == 'Luiz Celeste') & (model_df['course_id'] == 2)]

In [43]:
predict_df

,Unnamed: 0,course_id,cumgame_coeff,intercept,mse,n_value,nomatch,player,precip_coeff,prediction_score,wind_coeff
182,182,2,-0.0,55.285714,6.782129,21.0,NaN,Luiz Celeste,-0.395733,0.100205,0.651328


In [68]:
player1 = 'Luiz_Celeste'
player2 = 'Christian_Eck'
course = 2

In [75]:
player1_ns = player1.replace('_',' ')
player2_ns = player2.replace('_',' ')
course = int(course)
player1_data = model_df.loc[(model_df['player'] == player1_ns) & (model_df['course_id'] == course)]
player2_data = model_df.loc[(model_df['player'] == player2_ns) & (model_df['course_id'] == course)]
player1_score = player1_data.mean()['intercept']
player2_score = player2_data.mean()['intercept']
player1_var = player1_data.mean()['mse']
player2_var = player2_data.mean()['mse']
score_diff = player2_score - player1_score
score_var = player1_var + player2_var
score_se = np.sqrt(score_var)
score_z = score_diff / score_se
odds = st.norm.cdf(score_z)

list2 = []
list2.append(player1_score)
list2.append(player2_score)
list2.append(odds)

In [76]:
list2

[55.28571428571429, 49.56, 0.07184940540370478]

In [77]:
player1_data

,Unnamed: 0,course_id,cumgame_coeff,intercept,mse,n_value,nomatch,player,precip_coeff,prediction_score,wind_coeff
182,182,2,-0.0,55.285714,6.782129,21.0,NaN,Luiz Celeste,-0.395733,0.100205,0.651328


In [78]:
player2_data

,Unnamed: 0,course_id,cumgame_coeff,intercept,mse,n_value,nomatch,player,precip_coeff,prediction_score,wind_coeff
461,461,2,-0.652565,49.56,8.552478,25.0,NaN,Christian Eck,0.163612,0.263124,0.911427


In [5]:
timings_df = pd.read_csv('../../../ScoreCalculator/data/course_data.csv')
timings_df.head()

,course,lat,lon,dst,hour-6,hour-7,hour-8,hour-9,hour-10,hour-11,hour-12,hour-13,hour-14,hour-15,hour-16,hour-17,hour-18,hour-19,hour-20,hour-21
0,0,37.770651,-122.490969,all,0,0,0.2,0.600,1.0000,0.8,0.4,0.000,0,0.000,0.000,0.0,0,0.0,0.0,0
1,1,37.743375,-122.120676,all,0,0,0.0,0.333,0.6670,1.0,1.0,1.000,1,0.667,0.333,0.0,0,0.0,0.0,0
2,2,37.860605,-122.301984,no,0,0,0.0,0.000,0.6667,1.0,1.0,0.333,0,0.000,0.000,0.0,0,0.0,0.0,0
3,2,37.860605,-122.301984,yes,0,0,0.0,0.000,0.0000,0.0,0.0,0.000,0,0.000,0.200,0.6,1,0.8,0.4,0


In [6]:
timings_df.loc[timings_df.course == 1].values

array([[1, 37.7433753, -122.120676, 'all', 0, 0, 0.0,
        0.33299999999999996, 0.667, 1.0, 1.0, 1.0, 1, 0.667,
        0.33299999999999996, 0.0, 0, 0.0, 0.0, 0]], dtype=object)

In [24]:
timings_df.loc[(timings_df.course == 2) & ((timings_df.dst == 'yes') | (timings_df.dst == 'all'))].values[0][4:]

array([0, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0, 0.0, 0.2, 0.6, 1, 0.8, 0.4,
       0], dtype=object)

In [21]:
test_date = dt.datetime.fromisoformat('2019-08-01')
test_date.timestamp()

1564642800.0

In [23]:
timings_df.course.dtype

dtype('int64')